# ADAPT Pro - Topic 3 - Automation, Visualization and Best Practices

# Advanced Visualizations and Dashboards
This notebook showcases advanced applications of:
- plotly package for interactive visualizations
- perspective package for interacting with DataFrame tables
- ipywidgets for creating interactive components for dashbaords


## Perspective Widget
- `PerspectiveWidget` is a Jupyter plugin from the perspective package that offers functionality to interact with data sets such:
    - dynamic filtering and sorting
    - dynamic groupby (creating pivots/aggregations of data)
    - creating interactive charts such as bar charts, line graphs, etc.
    - exporting filtered data as CSV files

In [1]:
import pandas as pd
from perspective import PerspectiveWidget

In [2]:
finData = pd.read_excel("ADAPT2021/ExData/Data Manipulation Worksheet.xlsx", sheet_name="Financing Table Clean")
finData.head()

In [3]:
PerspectiveWidget(finData)

## Visualizations with Plotly

In [4]:
import plotly.express as px

### Simple visualizations recap

In [5]:
df = pd.read_csv('ADAPT2021/StockData/AAPL.csv', parse_dates=['Date'], index_col=['Date'])

In [19]:
fig = px.line(df, y="Close", title='Apple')
fig.show()

In [12]:
figBar = px.bar(finData, x='INDUSTRY', y='SIZE', color='TYPE', hover_name="ISSUER",
                title="Total Deal Value by Industry",
                width=900,height=600)
figBar.show()

### Advanced Visualizations - Formatting Settings

Extra arguments that are useful to add to the plotting formulas:
- facet_col - break the chart into subplots based on a category into multiple columns
- facet_row - break the chart into subplots based on a category into multiple rows
- facet_col_wrap - # of columns to show in a grid if too many columns to display
- width, height - to change the sizing of graph
- color_discrete_sequence - colors to use in the graph for different series, can provide a list of English words (e.g. `['darkblue','lightblue']`)
- color_discrete_map - same as above but can provide a mapping with a dictionary of the categories (e.g. `{'JPM':"darkblue", 'GS':"lightblue"}`)

#### Facet_Col example - Side by Side Comparison of Types of Deals by Sector

In [33]:
finData['TYPE'].unique()

In [36]:
debtFilters = ['Bank Notes','Sr Notes','CD Notes', 'Global Notes','Notes','Bonds']
equityFilters = ['Initial Public Offering','Equity Follow-On Offering','Preferred Securities']
debtDeals = finData[finData['TYPE'].isin(debtFilters)]
equityDeals = finData[finData['TYPE'].isin(equityFilters)]

In [42]:
figDebt = px.bar(debtDeals, x='INDUSTRY', y='SIZE', facet_col='TYPE', facet_col_wrap=3,
                color='INDUSTRY', hover_name="ISSUER",
                title=" Side by Side Comparison of Types of Deals by Sector",
                width=900,height=600)
figDebt.show()

In [43]:
figEquity = px.bar(equityDeals, x='INDUSTRY', y='SIZE', facet_col='TYPE', facet_col_wrap=3,
                color='INDUSTRY', hover_name="ISSUER",
                title=" Side by Side Comparison of Types of Deals by Sector",
                width=900,height=500)
figEquity.show()

#### Facet_Col example - Side by Side Comparison of Lead Underwriters

In [45]:
banks = ['J.P. Morgan','Goldman Sachs']
filteredData = finData[finData['LEAD UNDERWRITER'].isin(banks)]

In [46]:
figBanks = px.bar(filteredData, x='INDUSTRY', y='SIZE', facet_col='LEAD UNDERWRITER', color='TYPE', hover_name="ISSUER",
                title="Total Deal Value by Industry and Type",
                width=900,height=600)
figBanks.show()

#### Facet_Row example - Time Series of Different Stocks

In [47]:
tickers = ['JPM','GS']
tables = []
for ticker in tickers:
    temp_df = pd.read_csv('ADAPT2021/StockData/'+ticker+'.csv', parse_dates=['Date'], index_col=['Date'])
    temp_df['Ticker'] = ticker
    tables.append(temp_df)

tickerData = pd.concat(tables)

In [55]:
fig = px.line(tickerData, y="Close", color='Ticker', title='Ticker Share Prices',
             color_discrete_map={'JPM':"darkblue", 'GS':"lightblue"})
fig.show()

#can also use a dictionary: color_discrete_map={'JPM':"darkblue", 'GS':"lightblue"}
#or list: color_discrete_sequence=["darkblue", "lightblue"]

In [49]:
fig = px.line(tickerData, y="Close", facet_row='Ticker', title='Ticker Share Prices',height=600)
fig.show()

### Advanced Visualizations - Trendlines
- one of the advanced features of plotly is to add trendlines to scatter plots such as:
    - simple and exponential moving averages (without having to manually calculate with `.rolling()` in pandas)
    - regression lines (e.g. OLS without having to run statsmodels package)
- additional examples: https://plotly.com/python/linear-fits/

#### Linear Regression with OLS

In [119]:
sp500 = pd.read_csv('ADAPT2021/StockData/SP500.csv', parse_dates=['Date'], index_col=['Date'])
aapl = pd.read_csv('ADAPT2021/StockData/AAPL.csv', parse_dates=['Date'], index_col=['Date'])
sp500['Returns'] = sp500['Close'].pct_change()
aapl['Returns'] = aapl['Close'].pct_change()
olsData = sp500.merge(aapl, how='inner',on=['Date'], suffixes=('_SP','_AAPL'))
olsData = olsData.dropna()

In [143]:
figOLS = px.scatter(olsData, x="Returns_SP", y="Returns_AAPL", trendline="ols", trendline_color_override="red")
figOLS.show()

OLS results can also be extracted directly from the grapy

In [77]:
results = px.get_trendline_results(fig)
print(results)

In [153]:
results.px_fit_results.iloc[0].summary()

#### Moving Averages

In [56]:
nke = pd.read_csv('ADAPT2021/StockData/NKE.csv', parse_dates=['Date'], index_col=['Date'])

In [65]:
fig = px.scatter(nke, y="Close", trendline="rolling", trendline_options=dict(window=250),
                title="Nike 250-day moving average")
fig.show()

Can also change the function used and color settings with `trendline_options` and `trendline_color_override`

In [71]:
fig = px.scatter(nke, y="Close", trendline="expanding", trendline_options=dict(function="max"),
                title="Expanding Maximum", trendline_color_override="red")
fig.show()

### Advanced Visualizations - Animations
- can show changes over time with `animation_frame=Category`, where category could be a column that keeps track of year, month, etc.
- more examples: https://plotly.com/python/animations/

In [80]:
data['Year'] = data.index.year

In [84]:
px.scatter(data, x="Returns_SP", y="Returns_AAPL", animation_frame="Year",
           trendline="ols", trendline_color_override="red",
           height=600)

### Advanced Visualizations - Adding Traces and Annotations
- annotations and additional series can be added to a simple plotly express figure using:
    - `fig.add_shape()` - for custom shapes
    - `fig.add_vline()` or `fig.add_hline()` - for a custom vertical or horizontal line on chart
    - `fig.add_annotation()` - for custom text
- more examples: 
    - Custom shapes: https://plotly.com/python/horizontal-vertical-shapes/
    - Custom annotations: https://plotly.com/python/text-and-annotations/

In [87]:
maxPrice = aapl['Close'].max()
minPrice = aapl['Close'].min()
avgPrice = aapl['Close'].mean()

In [116]:
fig = px.line(aapl, y='Close', title='Apple Share Price', height=500)

#adding horiztonal dotted lines for max and min
fig.add_hline(y=minPrice, line_width=3, line_dash="dot", line_color="red")

fig.add_hline(y=maxPrice, annotation_text="Max Price: ${:.2f}".format(maxPrice),
              annotation_position="top right",
             line_width=3, line_dash="dash", line_color="green")

#custom formatting of annotations
fig.update_annotations(font=dict(family="sans serif", size=18, color="green"))
fig.add_annotation(x=aapl.index.min(), y=minPrice,text="Min Price: ${:.2f}".format(minPrice),
                   arrowhead=1, showarrow=True,
                  font=dict(family="sans serif", size=18, color="red"))

#Vertical shaded region
fig.add_vrect(x0="2015-07-20", x1="2016-2-11", 
              annotation_text="Decline", annotation_position="top left",
              fillcolor="red", opacity=0.25, line_width=0)

fig.show()

## Multiple Outputs with Plotly
- instead of rendering the plotly express graphs inside a Jupyter file, they can be exported as stand-alone HTML files
- multiple figure objects along with text and tables can be added to the exported HTML file

In [118]:
import plotly.graph_objects as go

In [156]:
#Outputs
summaryOLS = results.px_fit_results.iloc[0].summary()
# summaryOLS

In [166]:
#Table output
olsData['Date'] = olsData.index.date 
outputData = olsData[['Date','Close_SP','Close_AAPL','Returns_SP','Returns_AAPL']]
outputData.head()

In [169]:
figT = go.Figure(data=[go.Table(
                        header=dict(
                            values=list(outputData.columns),
                            fill_color='darkblue',
                            font=dict(size=15, color='white'),
                            align="right"
                        ),
                        cells=dict(
                            values=[outputData[k].tolist() for k in outputData.columns],
                            align = "right",
                            format=['None','$,.2f','$,.2f','.2%','.2%']
                        )
                    )])
# figT

In [148]:
# figOLS #OLS Chart
figSP500 = px.line(sp500, y='Close',title='S&P 500')
figAAPL = px.line(aapl, y='Close',title='Apple Inc')
# figSP500
# figAAPL

#### Exporting HTML File

In [177]:
text = '''
<h1> Demo of Plotly Export to HTML</h1>
<p>Multiple figures and components can be exported at the same time:</p>
<ul>
<li>plotly express figures</li>
<li>custom HTML code (for text and tables)</li>
<li>pandas DataFrame tables converted to HTML with .to_html()</li>
<li>more complex formatted tables with plotly</li>
'''

In [178]:
figures = [figOLS, figT, figAAPL, figSP500]
with open('plotlyDemo.html','a') as f:
    f.write(text)
    f.write(summaryOLS.as_html())
    for output in figures:        
        f.write(output.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(outputData.head().to_html())